In [3]:
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv("Final_dataset.csv")


In [5]:
df.head()

,Unnamed: 0,Date flow,Dst IP Addr:Port,Durat,Flags,Flows,Label,Labels,Packets Bytes,Prot,Src IP Addr:Port,Tos,start time
0,0,2011-08-15,205.188.10.203:443,0.000,A_,60,1,LEGITIMATE,1,TCP,147.32.84.134:45266,0,11:00:05.704
1,1,2011-08-15,147.32.84.118:51463,4.996,PA_,84327,1,Background,58,TCP,62.44.1.18:80,0,11:00:05.704
2,2,2011-08-15,62.44.1.18:80,4.892,A_,1740,1,Background,29,TCP,147.32.84.118:51463,0,11:00:05.706
3,3,2011-08-15,147.32.85.103:18006,0.000,INT,175,1,Background,1,UDP,147.32.80.9:53,0,11:00:05.707
4,4,2011-08-15,147.32.80.9:53,0.000,INT,71,1,Background,1,UDP,147.32.85.103:41094,0,11:00:05.709


In [6]:
df.columns

Index(['Unnamed: 0', 'Date flow', 'Dst IP Addr:Port', 'Durat', 'Flags',
       'Flows', 'Label', 'Labels', 'Packets Bytes', 'Prot', 'Src IP Addr:Port',
       'Tos', 'start time'],
      dtype='object')

In [7]:
df.drop(labels = [ "Src IP Addr:Port" , 'start time' , 'Label' , 'Tos','Unnamed: 0','Date flow'] ,axis = 1 , inplace = True)
df.head()

,Dst IP Addr:Port,Durat,Flags,Flows,Labels,Packets Bytes,Prot
0,205.188.10.203:443,0.000,A_,60,LEGITIMATE,1,TCP
1,147.32.84.118:51463,4.996,PA_,84327,Background,58,TCP
2,62.44.1.18:80,4.892,A_,1740,Background,29,TCP
3,147.32.85.103:18006,0.000,INT,175,Background,1,UDP
4,147.32.80.9:53,0.000,INT,71,Background,1,UDP


In [8]:
df['Labels'].value_counts()

Background    3895469
LEGITIMATE     336103
Botnet           6472
Name: Labels, dtype: int64

In [9]:
botnet_dataset = df[df['Labels']=='Botnet']
botnet_dataset

,Dst IP Addr:Port,Durat,Flags,Flows,Labels,Packets Bytes,Prot
1812,147.32.84.165:1087,0.000,R_,60,Botnet,1,TCP
1872,208.73.210.29:80,0.589,SPA_,412,Botnet,5,TCP
1909,208.73.210.29:80,0.401,_FSPA,1883,Botnet,5,TCP
10373,147.32.84.165:1089,0.000,R_,60,Botnet,1,TCP
25537,147.32.84.165:1052,4.867,FA_,300,Botnet,5,TCP
...,...,...,...,...,...,...,...
4238039,147.32.96.69,0.000,NaN,1066,Botnet,1,ICMP
4238040,147.32.96.69,0.000,NaN,1066,Botnet,1,ICMP
4238041,147.32.96.69,0.000,NaN,1066,Botnet,1,ICMP
4238042,147.32.96.69,0.000,NaN,1066,Botnet,1,ICMP


In [10]:
indexnames = df[df['Labels']=='Botnet'].index
df.drop(indexnames,inplace = True )

In [15]:
df1 = df.head(6472)

In [16]:
x_bg = df1.loc[:, df1.columns != 'Labels']
y_bg = df1.loc[:, df1.columns == 'Labels']
y_bg

,Labels
0,LEGITIMATE
1,Background
2,Background
3,Background
4,Background
...,...
6470,Background
6471,Background
6472,Background
6473,Background


In [18]:
x_botnet = botnet_dataset.loc[:, botnet_dataset.columns != 'Labels']
y_botnet = botnet_dataset.loc[:, botnet_dataset.columns == 'Labels']

In [19]:
print(x_botnet.shape , "bg" , x_bg.shape)

(6472, 6) bg (6472, 6)


In [32]:

import numpy as np
  
y_botnet = np.full((6472 , 1 ) , 1)
y_bg = np.full((6472 , 1 ) , 0)

In [30]:
y_botnet

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [21]:
print(y_botnet)
print(y_bg)

[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [31]:
y_botnet = pd.DataFrame( y_botnet.flatten())
y_botnet.head()

,0
0,1
1,1
2,1
3,1
4,1


In [34]:
y_botnet = pd.DataFrame(y_botnet.flatten() )
print(y_botnet)
y_bg = pd.DataFrame(y_bg.flatten() )
print(y_bg)

      0
0     1
1     1
2     1
3     1
4     1
...  ..
6467  1
6468  1
6469  1
6470  1
6471  1

[6472 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
6467  0
6468  0
6469  0
6470  0
6471  0

[6472 rows x 1 columns]


In [40]:
print(y_bg)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
6467  0
6468  0
6469  0
6470  0
6471  0

[6472 rows x 1 columns]


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

xtrain_botnet, xtest_botnet, ytrain_botnet, ytest_botnet = train_test_split(x_botnet,y_botnet,test_size=0.2,random_state=45)

In [43]:
print(xtrain_botnet.shape)
print(ytrain_botnet.shape)
print(xtest_botnet.shape)
print(ytest_botnet.shape)


(5177, 6)
(5177, 1)
(1295, 6)
(1295, 1)


In [44]:
xtrain_bg, xtest_bg, ytrain_bg, ytest_bg = train_test_split(x_bg,y_bg,test_size=0.2,random_state=45)

In [48]:
print(xtrain_bg.shape)
print(ytrain_bg.shape)
print(xtest_bg.shape)
print(ytest_bg.shape)
print(xtrain_botnet.shape)
print(ytrain_botnet.shape)
print(xtest_botnet.shape)
print(ytest_botnet.shape)

(5177, 6)
(5177, 1)
(1295, 6)
(1295, 1)
(5177, 6)
(5177, 1)
(1295, 6)
(1295, 1)


In [46]:
x_train = xtrain_bg.append(xtrain_botnet)
print(x_train)


             Dst IP Addr:Port  Durat Flags  Flows  Packets Bytes  Prot
171      194.108.204.27:44728  4.974   PA_   1444              4   TCP
5028           147.32.80.9:53  0.000   INT     89              1   UDP
6071       147.102.3.36:45035  0.000   INT    179              1   UDP
3020       24.59.84.197:36116  0.000   INT     71              1   UDP
5606      147.32.84.229:13363  0.000   INT     77              1   UDP
...                       ...    ...   ...    ...            ...   ...
1177319      85.190.0.3:41426  0.000   RA_     60              1   TCP
3000282     147.32.84.165:808  0.000    S_     74              1   TCP
2933942          147.32.96.69  0.000   ECO   1066              1  ICMP
4082604    147.32.84.165:2721  0.000   INT    141              1   UDP
3979036       24.71.223.11:25  3.004    S_    124              2   TCP

[10354 rows x 6 columns]


In [49]:
x_train = xtrain_bg.append(xtrain_botnet)
print(x_train)
x_test = xtest_bg.append(xtest_botnet)
print(x_test)

             Dst IP Addr:Port  Durat Flags  Flows  Packets Bytes  Prot
171      194.108.204.27:44728  4.974   PA_   1444              4   TCP
5028           147.32.80.9:53  0.000   INT     89              1   UDP
6071       147.102.3.36:45035  0.000   INT    179              1   UDP
3020       24.59.84.197:36116  0.000   INT     71              1   UDP
5606      147.32.84.229:13363  0.000   INT     77              1   UDP
...                       ...    ...   ...    ...            ...   ...
1177319      85.190.0.3:41426  0.000   RA_     60              1   TCP
3000282     147.32.84.165:808  0.000    S_     74              1   TCP
2933942          147.32.96.69  0.000   ECO   1066              1  ICMP
4082604    147.32.84.165:2721  0.000   INT    141              1   UDP
3979036       24.71.223.11:25  3.004    S_    124              2   TCP

[10354 rows x 6 columns]
             Dst IP Addr:Port  Durat  Flags  Flows  Packets Bytes  Prot
1539      90.187.47.244:55056  4.150    INT   1178

In [51]:
y_train = ytrain_bg.append(ytrain_botnet)
print(y_train)
y_test = ytest_bg.append(ytest_botnet)
print(y_test)

      0
171   0
5025  0
6068  0
3017  0
5603  0
...  ..
163   1
4703  1
3616  1
6012  1
5763  1

[10354 rows x 1 columns]
      0
1539  0
3107  0
355   0
3425  0
1255  0
...  ..
2945  1
5805  1
3796  1
4053  1
5073  1

[2590 rows x 1 columns]


In [55]:
x_train.to_csv( "X_Train_Dataset.csv")
y_train.to_csv( "Y_Train_Dataset.csv")
x_test.to_csv(  "X_Test_Dataset.csv")
y_test.to_csv( "Y_Test_Dataset.csv")

# MODEL

In [1]:
import pandas as pd


In [2]:
x_train = pd.read_csv( "X_Train_Dataset.csv")
y_train = pd.read_csv( "Y_Train_Dataset.csv")
x_test = pd.read_csv(  "X_Test_Dataset.csv")
y_test = pd.read_csv( "Y_Test_Dataset.csv")

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10354, 7)
(10354, 2)
(2590, 7)
(2590, 2)


In [4]:
y_train

,Unnamed: 0,0
0,171,0
1,5025,0
2,6068,0
3,3017,0
4,5603,0
...,...,...
10349,163,1
10350,4703,1
10351,3616,1
10352,6012,1


In [5]:
x_train

,Unnamed: 0,Dst IP Addr:Port,Durat,Flags,Flows,Packets Bytes,Prot
0,171,194.108.204.27:44728,4.974,PA_,1444,4,TCP
1,5028,147.32.80.9:53,0.000,INT,89,1,UDP
2,6071,147.102.3.36:45035,0.000,INT,179,1,UDP
3,3020,24.59.84.197:36116,0.000,INT,71,1,UDP
4,5606,147.32.84.229:13363,0.000,INT,77,1,UDP
...,...,...,...,...,...,...,...
10349,1177319,85.190.0.3:41426,0.000,RA_,60,1,TCP
10350,3000282,147.32.84.165:808,0.000,S_,74,1,TCP
10351,2933942,147.32.96.69,0.000,ECO,1066,1,ICMP
10352,4082604,147.32.84.165:2721,0.000,INT,141,1,UDP


In [6]:
x_train_1 = x_train.drop(labels = [ "Dst IP Addr:Port" , "Unnamed: 0"] ,axis = 1 )
x_test_1 = x_test.drop(labels = [ "Dst IP Addr:Port" , "Unnamed: 0"] ,axis = 1 )
y_train = y_train.drop(labels = [ "Unnamed: 0"] ,axis = 1 )
y_test = y_test.drop(labels = [  "Unnamed: 0"] ,axis = 1 )

In [7]:
x_train_1['Flags'].describe

<bound method NDFrame.describe of 0        PA_
1        INT
2        INT
3        INT
4        INT
        ... 
10349    RA_
10350     S_
10351    ECO
10352    INT
10353     S_
Name: Flags, Length: 10354, dtype: object>

In [8]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
x_train_1['Flags']= label_encoder.fit_transform(x_train_1['Flags'].astype(str)) 
  
x_train_1['Flags'].unique() 



array([29, 16,  1, 44, 41,  8, 58,  5,  7, 13, 38, 45, 31, 12, 55, 14, 57,
       53, 11,  9, 37, 32, 42,  2, 10,  6, 25, 49, 34, 52, 59, 54, 50, 33,
       35, 19, 17, 18, 48, 47, 36, 23,  4, 60, 21, 22, 20, 46,  0, 43, 26,
       30, 40, 28,  3, 24, 39, 15, 56, 51, 27], dtype=int64)

In [9]:
x_train_1

,Durat,Flags,Flows,Packets Bytes,Prot
0,4.974,29,1444,4,TCP
1,0.000,16,89,1,UDP
2,0.000,16,179,1,UDP
3,0.000,16,71,1,UDP
4,0.000,16,77,1,UDP
...,...,...,...,...,...
10349,0.000,31,60,1,TCP
10350,0.000,45,74,1,TCP
10351,0.000,5,1066,1,ICMP
10352,0.000,16,141,1,UDP


In [10]:
x_train_1['Prot']= label_encoder.fit_transform(x_train_1['Prot'].astype(str)) 
  
x_train_1['Prot'].unique() 


array([6, 7, 2, 0, 5, 4, 3, 1], dtype=int64)

In [11]:
x_test_1['Flags']= label_encoder.fit_transform(x_test_1['Flags'].astype(str)) 
  
x_test_1['Flags'].unique() 

x_test_1['Prot']= label_encoder.fit_transform(x_test_1['Prot'].astype(str)) 
  
x_test_1['Prot'].unique() 

array([5, 4, 1, 3, 2, 0], dtype=int64)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [13]:
descision_tree_model = DecisionTreeClassifier()
descision_tree_model.fit(x_train_1,y_train)
prediction = descision_tree_model.predict(x_test_1)
print('Decision Accuracy Score:',round(accuracy_score(y_test,prediction)*100),'%')

Decision Accuracy Score: 72.0 %


In [14]:
multinomial_naive_bayes = GaussianNB()
multinomial_naive_bayes.fit(x_train_1,y_train)
prediction_naive = multinomial_naive_bayes.predict(x_test_1)
print('Naive Bayes Accuracy Score:',round(accuracy_score(y_test,prediction_naive)*100),'%')

Naive Bayes Accuracy Score: 49.0 %


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
Random_forest_model = RandomForestClassifier(class_weight='balanced')
Random_forest_model.fit(x_train_1,y_train.ravel())
prediction_Random_forest_model = Random_forest_model.predict(x_test_1)
print('Random Forest Accuracy Score:',round(accuracy_score(y_test.ravel(),prediction_Random_forest_model)*100),'%')

AttributeError: 'DataFrame' object has no attribute 'ravel'

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='radial')
svm.fit(x_train_1 , y_train.to_numpy().ravel())
svm_predict = svm.predict(x_test_1)
print('support vector  Score:',round(accuracy_score(y_test.to_numpy().ravel(),svm_predict)*100),'%')

In [16]:
Logistic_model = LogisticRegression(C=1000)
Logistic_model.fit(x_train_1,y_train)
prediction_Logistic = Logistic_model.predict(x_test_1)
print('Logistic Regression Accuracy Score:',round(accuracy_score(y_test,prediction_Logistic)*100),'%')

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression Accuracy Score: 71.0 %
